In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\Nisarg\\OneDrive\\Desktop\\7 Project\\Medical-Assistant-main\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Nisarg\\OneDrive\\Desktop\\7 Project\\Medical-Assistant-main'

In [5]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract data 
def load_pdf(data):
    loader=DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf(data=r'C:\Users\Nisarg\OneDrive\Desktop\7 Project\Medical-Chatbot\Data')

FileNotFoundError: Directory not found: 'C:\Users\Nisarg\OneDrive\Desktop\7 Project\Medical-Chatbot\Data'

In [ ]:
#split Data 
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print("Length of Text chunks",len(text_chunks))

Length of Text chunks 0


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
#Download Embedding from hgging face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings=download_hugging_face_embeddings()

C:\Users\Nisarg\AppData\Local\Temp\ipykernel_2240\259317463.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
query_result=embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
Ollama_API_KEY=os.environ.get('Ollama_API_KEY')



In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_2CMN9z_QQWYTdsd89TEGCU19dNRKUFA7D7Pqc5CtTfqqQMESw6otxhDG8vGKGa1UboCech")
index_name ="askmedi"
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
        
    )
)




{
    "name": "askmedi",
    "metric": "cosine",
    "host": "askmedi-2opht5t.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [ ]:
import os
os.environ["PINECONE_API_KEY"]="pcsk_2CMN9z_QQWYTdsd89TEGCU19dNRKUFA7D7Pqc5CtTfqqQMESw6otxhDG8vGKGa1UboCech"
os.environ["Ollama_API_KEY"]="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"

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [ ]:
retrieved_docs=retriever.invoke("what is acne?")

In [ ]:
retrieved_docs

[]

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaLLM
llm=OllamaLLM(model="llama3")



In [ ]:

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaLLM


In [ ]:
system_prom=(
    "You are an assistant for question-answering tasks"
    "Use the following pieces of retrived context to answer"
    "the question.If you don't know the answer,say thank you"
    "don't know.Use three sentence maximun and keep the"
    "answer concise."
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prom),
        ("human","{input}"),
    ]
)

In [ ]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)
rag_chain = RetrievalQA(llm=llm, retriever=retriever)


In [ ]:
response=llm.invoke("what is acne?")
print(response)

Acne is a common skin condition characterized by the formation of pimples, blackheads, and whiteheads. It occurs when the pores on the skin become clogged with dead skin cells, oil, and bacteria.

The main causes of acne are:

1. **Clogged pores**: When dead skin cells and oil (sebum) mix together in the pores, they can form a blockage that traps bacteria.
2. **Bacteria growth**: A type of bacteria called Propionibacterium acnes (P. acnes) is naturally found on the skin and can multiply quickly in clogged pores.
3. **Hormonal changes**: Hormonal fluctuations, such as those experienced during puberty, menstruation, pregnancy, or menopause, can lead to increased sebum production and acne.
4. **Stress**: Stress can increase the production of hormones like cortisol, which can contribute to acne.
5. **Genetics**: If your parents had acne, you may be more likely to develop it too.

Acne typically appears on the face, but it can also occur on other areas of the body, such as:

* Back
* Chest
